In [31]:
import json
import urllib.request
import os
import pandas
from pdfminer.high_level import extract_text
import traceback
import string

**COLLECT AI ARTICLES**

In [2]:
# Charger les IDs des articles à partir du fichier JSON
with open("arxiv-metadata-oai-snapshot.json", "r") as file:
    articles_ids = []
    for line in file:
        try:
            article = json.loads(line)
            id = article.get('id', '')
            categories = article.get('categories', '')
            
            # Vérifier si l'article appartient à la catégorie "cs.AI" (IA)
            if 'cs.AI' in categories:
                articles_ids.append(id)
        except json.JSONDecodeError as e:
            print(f"Erreur de décodage JSON : {e}")
            continue

In [3]:
print(articles_ids)

['0704.0047', '0704.0050', '0704.0304', '0704.0985', '0704.1028', '0704.1394', '0704.1409', '0704.1675', '0704.1676', '0704.1783', '0704.2010', '0704.2083', '0704.2201', '0704.3157', '0704.3359', '0704.3395', '0704.3433', '0704.3453', '0704.3515', '0704.3886', '0704.3905', '0705.0025', '0705.0197', '0705.0199', '0705.0588', '0705.0593', '0705.0693', '0705.0734', '0705.0760', '0705.0761', '0705.0969', '0705.1031', '0705.1110', '0705.1209', '0705.1244', '0705.1309', '0705.1617', '0705.1673', '0705.1999', '0705.2011', '0705.2235', '0705.2236', '0705.2305', '0705.2307', '0705.2310', '0705.2485', '0705.2516', '0705.2765', '0705.3360', '0705.3561', '0705.3766', '0705.4302', '0705.4566', '0705.4584', '0706.0022', '0706.0465', '0706.0585', '0706.1001', '0706.1137', '0706.1290', '0706.3639', '0706.4323', '0706.4375', '0707.0498', '0707.0701', '0707.0704', '0707.0705', '0707.0808', '0707.1452', '0707.2506', '0707.3030', '0707.3205', '0707.3457', '0707.3559', '0707.3781', '0707.3972', '0707.4289'

In [4]:
# URL de base pour télécharger les PDFs
base_url = "https://arxiv.org/pdf/"

# Liste des URL complètes des PDFs
pdf_urls = [base_url + id for id in articles_ids]

In [5]:
# Dossier de destination pour enregistrer les PDFs
output_folder = "AI_articles_pdfs"

# Vérifier si le dossier de destination existe, sinon le créer
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Télécharger les PDFs et les enregistrer dans le dossier de destination
for url in pdf_urls:
    file_name = url.split("/")[-1] + ".pdf"  # Ajouter l'extension .pdf au nom du fichier
    file_path = os.path.join(output_folder, file_name)
    
    try:
        urllib.request.urlretrieve(url, file_path)
        print(f"Téléchargement réussi : {file_name}")
    except Exception as e:
        print(f"Échec du téléchargement pour l'URL : {url}. Erreur : {str(e)}")

Téléchargement réussi : 0704.0047.pdf
Téléchargement réussi : 0704.0050.pdf
Téléchargement réussi : 0704.0304.pdf
Téléchargement réussi : 0704.0985.pdf
Téléchargement réussi : 0704.1028.pdf
Téléchargement réussi : 0704.1394.pdf
Téléchargement réussi : 0704.1409.pdf
Téléchargement réussi : 0704.1675.pdf
Téléchargement réussi : 0704.1676.pdf
Téléchargement réussi : 0704.1783.pdf
Téléchargement réussi : 0704.2010.pdf
Téléchargement réussi : 0704.2083.pdf
Téléchargement réussi : 0704.2201.pdf
Téléchargement réussi : 0704.3157.pdf
Téléchargement réussi : 0704.3359.pdf
Téléchargement réussi : 0704.3395.pdf
Téléchargement réussi : 0704.3433.pdf
Téléchargement réussi : 0704.3453.pdf
Téléchargement réussi : 0704.3515.pdf
Téléchargement réussi : 0704.3886.pdf
Téléchargement réussi : 0704.3905.pdf
Téléchargement réussi : 0705.0025.pdf
Téléchargement réussi : 0705.0197.pdf
Téléchargement réussi : 0705.0199.pdf
Téléchargement réussi : 0705.0588.pdf
Téléchargement réussi : 0705.0593.pdf
Téléchargeme

**CONTENT EXTRACTION**

In [29]:
# Chemin du dossier contenant les fichiers PDF
pdf_folder_path = "AI_articles_pdfs/"

extracted_texts = []
ids = []

# Itération à travers les fichiers PDF dans le dossier
for filename in os.listdir(pdf_folder_path):
    if filename.endswith(".pdf"):
        # Supprimer l'extension ".pdf" du nom de fichier pour obtenir l'ID
        id_number = filename.replace(".pdf", "")

        # Chemin complet du fichier PDF
        pdf_file_path = os.path.join(pdf_folder_path, filename)

        # Vérifie si le fichier existe
        if not os.path.exists(pdf_file_path):
            print(f"Le fichier {pdf_file_path} n'existe pas. Passage au fichier suivant.")
            continue

        try:
            # Extrait le texte du PDF
            extracted_text = extract_text(pdf_file_path)

            # Stocke l'ID et le texte extrait dans les listes correspondantes
            ids.append(id_number)
            extracted_texts.append(extracted_text)
            
        except Exception as e:
            print(f"Erreur lors de l'extraction du texte depuis {pdf_file_path}: {str(e)}")
            traceback.print_exc()

# Créer un DataFrame à partir des listes
import pandas as pd

df = pd.DataFrame({"ID": ids, "Texte Extrait": extracted_texts})

# Enregistrer le DataFrame dans un fichier CSV
df.to_csv("contenu_extracted_with_ids.csv", index=False, encoding="utf-8")

# Affiche un message lorsque l'extraction est terminée
print("Extraction terminée et le DataFrame a été enregistré dans contenu_extracted_with_ids.csv")

The PDF <_io.BufferedReader name='AI_articles_pdfs/0711.4309.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='AI_articles_pdfs/0802.3789.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Erreur lors de l'extraction du texte depuis AI_articles_pdfs/0803.1207.pdf: No /Root object! - Is this really a PDF?


Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_10424\221868021.py", line 23, in <module>
    extracted_text = extract_text(pdf_file_path)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pdfminer\high_level.py", line 168, in extract_text
    for page in PDFPage.get_pages(
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pdfminer\pdfpage.py", line 151, in get_pages
    doc = PDFDocument(parser, password=password, caching=caching)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pdfminer\pdfdocument.py", line 752, in __init__
    raise PDFSyntaxError("No /Roo

Erreur lors de l'extraction du texte depuis AI_articles_pdfs/0806.4511.pdf: No /Root object! - Is this really a PDF?


Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Temp\ipykernel_10424\221868021.py", line 23, in <module>
    extracted_text = extract_text(pdf_file_path)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pdfminer\high_level.py", line 168, in extract_text
    for page in PDFPage.get_pages(
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pdfminer\pdfpage.py", line 151, in get_pages
    doc = PDFDocument(parser, password=password, caching=caching)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\pc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pdfminer\pdfdocument.py", line 752, in __init__
    raise PDFSyntaxError("No /Roo

Extraction terminée et le DataFrame a été enregistré dans contenu_extracted_with_ids.csv


In [30]:
index_to_display = 2  # Remplacez par l'index que vous voulez afficher
print(extracted_texts[index_to_display])

0
1
0
2

t
c
O
3
1

]
T
I
.
s
c
[

3
v
4
0
3
0
.
4
0
7
0
:
v
i
X
r
a

The World as Evolving
Information

Carlos Gershenson1,2,3

1 Computer Sciences Department, Instituto de Investigaciones en
Matem´aticas Aplicadas y en Sistemas
Universidad Nacional Aut´onoma de M´exico1
Ciudad Universitaria, A.P. 20-726, 01000 M´exico D.F. M´exico
cgg@unam.mx http://turing.iimas.unam.mx/∼cgg
2 New England Complex Systems Institute
238 Main Street Suite 319 Cambridge, MA 02142, USA
3Centrum Leo Apostel, Vrije Universiteit Brussel
Krijgskundestraat 33 B-1160 Brussel, Belgium

This paper discusses the beneﬁts of describing the world as information, especially
in the study of the evolution of life and cognition. Traditional studies encounter prob-
lems because it is diﬃcult to describe life and cognition in terms of matter and energy,
since their laws are valid only at the physical scale. However, if matter and energy,
as well as life and cognition, are described in terms of information, evolution can be

In [32]:
# Charger le dataset CSV
df = pd.read_csv("contenu_extracted_with_ids.csv", encoding="utf-8")

# Fonction pour nettoyer les caractères non ASCII et indésirables
def clean_text(text):
    # Supprimer les caractères non ASCII
    cleaned_text = ''.join([char if ord(char) < 128 else ' ' for char in text])

    # Supprimer les caractères indésirables en remplaçant par des espaces
    unwanted_chars = ['\n', '\t', '\r']
    for char in unwanted_chars:
        cleaned_text = cleaned_text.replace(char, ' ')

    # Supprimer les espaces multiples
    cleaned_text = ' '.join(cleaned_text.split())
     # Convertir en minuscules
    text = text.lower()

    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    return cleaned_text

# Appliquer la fonction de nettoyage à la colonne "Texte Extrait"
df["Texte Extrait"] = df["Texte Extrait"].apply(clean_text)

# Enregistrer le dataset nettoyé
cleaned_csv_filename = "contenu_cleaned.csv"
df.to_csv(cleaned_csv_filename, index=False, encoding="utf-8")

print("Nettoyage terminé. Le fichier nettoyé a été enregistré sous le nom", cleaned_csv_filename)

Nettoyage terminé. Le fichier nettoyé a été enregistré sous le nom contenu_cleaned.csv


In [33]:
print(df.head())

         ID                                      Texte Extrait
0  704.0047  7 0 0 2 r p A 1 ] E N . s c [ 1 v 7 4 0 0 . 4 ...
1  704.0050  7 0 0 2 r p A 1 ] E N . s c [ 1 v 0 5 0 0 . 4 ...
2  704.0304  0 1 0 2 t c O 3 1 ] T I . s c [ 3 v 4 0 3 0 . ...
3  704.0985  Architecture for Pseudo Acausal Evolvable Embe...
4  704.1028  A Neural Network Approach to Ordinal Regressio...


In [34]:
# Chemin vers le fichier JSON
json_file_path = "arxiv-metadata-oai-snapshot.json"

# Liste pour stocker les objets JSON
data_list = []

# Ouvrir et charger les données JSON depuis le fichier
with open(json_file_path, "r", encoding="utf-8") as json_file:
    for line_number, line in enumerate(json_file, start=1):
        if line_number > 125939:
            break
        try:
            data = json.loads(line)
            data_list.append(data)
        except json.JSONDecodeError:
            pass

# Créer un DataFrame à partir des données extraites
df = pd.DataFrame(data_list)

# Afficher les premières lignes du DataFrame
print(df.head())

          id           submitter   
0  0704.0001      Pavel Nadolsky  \
1  0704.0002        Louis Theran   
2  0704.0003         Hongjun Pan   
3  0704.0004        David Callan   
4  0704.0005  Alberto Torchinsky   

                                             authors   
0  C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...  \
1                    Ileana Streinu and Louis Theran   
2                                        Hongjun Pan   
3                                       David Callan   
4           Wael Abu-Shammala and Alberto Torchinsky   

                                               title   
0  Calculation of prompt diphoton production cros...  \
1           Sparsity-certifying Graph Decompositions   
2  The evolution of the Earth-Moon system based o...   
3  A determinant of Stirling cycle numbers counts...   
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                  comments   
0  37 pages, 15 figures; published version  \
1    To appear in

In [35]:
selected_df = df[["id", "authors", "submitter", "title", "categories", "abstract"]]
selected_df = selected_df.rename(columns={"id": "ID"})
selected_df.to_csv("selected_information.csv", index=False)

# Charger les deux datasets
informations_df = pd.read_csv("selected_information.csv")
contenu_extracted_cleaned2_df = pd.read_csv("contenu_cleaned.csv")

# Fusionner les deux datasets selon l'ID en commun
merged_df = pd.merge(informations_df, contenu_extracted_cleaned2_df, on="ID", how="inner")

# Sauvegarder le nouveau dataset fusionné
merged_df.to_csv("final_dataset.csv", index=False)
print("Done!")

done


In [37]:
pd.read_csv("final_dataset.csv")

,ID,authors,submitter,title,categories,abstract,Texte Extrait
0,704.0047,T. Kosel and I. Grabec,Igor Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,The intelligent acoustic emission locator is...,7 0 0 2 r p A 1 ] E N . s c [ 1 v 7 4 0 0 . 4 ...
1,704.0050,T. Kosel and I. Grabec,Igor Grabec,Intelligent location of simultaneously active ...,cs.NE cs.AI,Part I describes an intelligent acoustic emi...,7 0 0 2 r p A 1 ] E N . s c [ 1 v 0 5 0 0 . 4 ...
2,704.0304,Carlos Gershenson,Carlos Gershenson,The World as Evolving Information,cs.IT cs.AI math.IT q-bio.PE,This paper discusses the benefits of describ...,0 1 0 2 t c O 3 1 ] T I . s c [ 3 v 4 0 3 0 . ...
3,704.0985,"Mohd Abubakr, R.M.Vinay",Mohd Abubakr,Architecture for Pseudo Acausal Evolvable Embe...,cs.NE cs.AI,Advances in semiconductor technology are con...,Architecture for Pseudo Acausal Evolvable Embe...
4,704.1028,Jianlin Cheng,Jianlin Cheng,A neural network approach to ordinal regression,cs.LG cs.AI cs.NE,Ordinal regression is an important type of l...,A Neural Network Approach to Ordinal Regressio...
...,...,...,...,...,...,...,...
585,905.3766,"Carmel Domshlak, Francesca Rossi, Kristen Bren...",Toby Walsh,Reasoning about soft constraints and condition...,cs.AI,Many real life optimization problems contain...,9 0 0 2 y a M 2 2 ] I A . s c [ 1 v 6 6 7 3 . ...
586,905.3769,"Alan M. Frisch, Ian Miguel, Zeynep Kiziltan, B...",Toby Walsh,Multiset Ordering Constraints,cs.AI,We identify a new and important global (or n...,Multiset Ordering Constraints Alan Frisch Dept...
587,905.3830,"F. Murtagh, A. Ganz, S. McKie, J. Mothe and K....",Fionn Murtagh,Tag Clouds for Displaying Semantics: The Case ...,cs.AI,We relate tag clouds to other forms of visua...,9 0 0 2 y a M 3 2 ] I A . s c [ 1 v 0 3 8 3 . ...
588,905.3885,"E. Elkind, P. Faliszewski, A. Slinko",Piotr Faliszewski,Swap Bribery,cs.GT cs.AI,"In voting theory, bribery is a form of manip...",9 0 0 2 y a M 4 2 ] T G . s c [ 1 v 5 8 8 3 . ...
